<h2>Facebook scrapper</h2>

One of the main problematics that we faced inthe project was the unability to access Facebook's closed groupes such as "On a mangé pour vous" with the graph api ( Facebook's data extraction API ). To face this problem we decided to create a Facebook scrapping script.

Our script uses selenium as a headless browser to access the closed groupes and extract the different posts and comments. This script was mainly inspired by <a href = "https://github.com/hikaruAi/FacebookBot">hikaruAi's FacebookBot</a> and adapt few methods from it such as the login and logout methods.

In [ ]:
from selenium import webdriver
from getpass import getpass
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import os
import csv 

Most Facebook scrappers go through the mbasic version of the website. This version is a light Facebook that is optimised for fast browsing and low bandwidth usage, thus, it's html and javascript codes are simpler than the Desktop version which makes data extraction easier. 

In [ ]:
def mfacebookToBasic(url):
    """(hikaruAi method)Reformat a url to load mbasic facebook instead of regular facebook, return the same string if
    the url don't contains facebook"""

    if "m.facebook.com" in url:
        return url.replace("m.facebook.com", "mbasic.facebook.com")
    elif "www.facebook.com" in url:
        return url.replace("www.facebook.com", "mbasic.facebook.com")
    else:
        return url

FacebookScrapper is the main class in this script. it extends the PhantomJS webdriver.

In [ ]:
class FacebookScrapper(webdriver.PhantomJS):
    

    def __init__(self):
        # pathToPhantomJs ="
        """(hikaruAi method) relativePhatomJs = "\\phantomjs.exe"
        service_args = None
        if images == False:
            service_args = ['--load-images=no', ]
        if pathToPhantomJs == None:
            path = self, os.getcwd() + relativePhatomJs
        else:
            path = pathToPhantomJs
            webdriver.PhantomJS.__init__(self, path, service_args=service_args)
        """
        
        dcap = dict(DesiredCapabilities.PHANTOMJS)
        dcap["phantomjs.page.settings.userAgent"] = (
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53 "
            "(KHTML, like Gecko) Chrome/15.0.87"
        )
        webdriver.PhantomJS.__init__(self, executable_path="C:\\Users\\Ska\\AppData\\Roaming\\npm\\node_modules\\phantomjs-prebuilt\\lib\\phantom\\bin\\phantomjs.exe",desired_capabilities=dcap)
        
        self.set_window_size (1000,500)
    def get(self, url):
        """(hikaruAi method )The make the driver go to the url but reformat the url if is for facebook page"""
        super().get(mfacebookToBasic(url))
       

    def login(self, email, password):
        """(hikaruAi method )Log to facebook using email (str) and password (str)"""

        url = "https://mbasic.facebook.com"
        self.get(url)
        email_element = self.find_element_by_name("email")
        email_element.send_keys(email)
        pass_element = self.find_element_by_name("pass")
        pass_element.send_keys(password)
        pass_element.send_keys(Keys.ENTER)
        if self.find_element_by_class_name("bi"):
            self.find_element_by_class_name("bp").click();        
        try:
            self.find_element_by_name("xc_message")
            print("Logged in")
            return True
        except NoSuchElementException as e:
            print("Fail to login : either connection problem or you are already logged in")
            return False

    def logout(self):
        """Log out from Facebook"""

        url = "https://mbasic.facebook.com/logout.php?h=AffSEUYT5RsM6bkY&t=1446949608&ref_component=mbasic_footer&ref_page=%2Fwap%2Fhome.php&refid=7"
        try:
            self.get(url)
            return True
        except Exception as e:
            print("Failed to log out ->\n", e)
            return False

<h4>getReviews:</h4> this function is mainly used to extract reviews their url and the date at which they were posted from Facebook Pages.

In [ ]:
def getReviews(bot,url) :
    bot.get(url)
    # list initialisation
    links_list =[]
    dates_list = []
    posts_list = []
    # we extract the links to each review by searching for <a> tags. Here we noticed that in every review's
    # link contains the 'activity' string, therefore, we check if the link contains this string.
    for a in bot.find_elements_by_tag_name('a'):
        if 'activity' in a.get_attribute('href'):
            links_list.append (a.get_attribute('href'))

    links_list=list ( set(links_list) )
    
    for l in links_list :
        # we go into every extracted link
        bot.get(l)
        try : 
            # a review's text is the only one inside the <p>
            review = bot.find_element_by_tag_name('p').text
            #the review's date is contained in the first <abbr> tag
            date = bot.find_elements_by_tag_name('abbr')[0].text
            posts_list.append(review)
            dates_list.append(date)

        except Exception as e:
            #in case of failure we append an empty string to keep matching indexes between the links_list and the otehr 2 lists 
            posts_list.append('')
            dates_list.append('')

    return posts_list, links_list, dates_list

<h4>GetPostInGroup:</h4> The function that extracts post from Facebook groups the urls to their comments and their posting date. This function is a deeply modified and more efficient version of HikaruAi's and unlike the older version it succeeds at extracting all posts. Given a URL and a depth it searchs a pages and extracts all the post. The moreText argument allows it to find the link to the next batch of posts note that <strong>this String changes along your facebook's language setting</strong>.

In [ ]:
def getPostInGroup(bot, url, deep=10, moreText="Show more"):
        
        # we go into the group's url
        bot.get(url)
        posts = []
        comments_links = []
        dates=[]
        for n in range(deep):
            # the main loop of post extraction, it goes through the posts following until it reaches depth deep
            print("Searching, deep ",n)
            # every post's id in a group has the 'u_0_' string in it, for this reason we use the xpath to extract them
            posts_element_list = bot.find_elements_by_xpath("//*[contains(@id,'u_0_')]")
            
            for p in posts_element_list :
                
                try:

                    text= p.find_element_by_tag_name("p").text
                    #we extract the link to the comments by looking for the "Comment" substring in the html code 
                    linkToComment = p.find_element_by_partial_link_text('Comment').get_attribute('href')
                    date = p.find_element_by_tag_name("abbr").text

                    if text not in posts:
                        # the use of the "//*[contains(@id,'u_0_')]" extracts every link twice. and since we didn't find
                        # a more efficient trick to extract every single comment we decided to just check if the post
                        # was already extracted
                        posts.append(text)
                        comments_links.append(linkToComment)
                        dates.append(date)
                except Exception:
                    continue

            try:
                # once this packet extraction is done we go into the next depth by going into the moreText's url
                more = bot.find_element_by_partial_link_text(
                    moreText).get_attribute('href')
                bot.get(more)
            except Exception as e:
                print(" Can't get more posts")
                break
        print ( posts)
        return posts,comments_links,dates

<h4>getCommentsInPost:</h4> This function is quite similar to getPostInGroup. Here we focus on extracting all the comments and their date.

In [ ]:
def getCommentsInPost(bot, url, deep=10, moreText="View previous comments…"):

        bot.get(url)    
        comments = []
        dates = []
        for n in range(deep):
            if n >=0 :
                print("Searching, deep ",n)
                # To extract comments we first look for the <h3> tags that envelop the commenter's name.
                h_list = bot.find_elements_by_tag_name('h3')
                for h in h_list :
                    try :
                        # the element containing the comment is the either first or second sibling of the commenter's 
                        # name element for this reason we go through the following code twice and add a try statement
                        c = h.find_elements_by_xpath('following-sibling::div[1]')
                        for comment in c :
                            try: 
                                date = h.find_element_by_xpath("..").find_element_by_tag_name("abbr").text
                                comments.append(comment.text)
                                dates.append ( date )
                                print ( comment.text )
                            except Exception :
                                continue

                        c = h.find_elements_by_xpath('following-sibling::div[2]')
                        for comment in c :
                            try:
                                date = h.find_element_by_xpath("..").find_element_by_tag_name("abbr").text
                                comments.append(comment.text)
                                dates.append(date)
                                print ( comment.text )   
                            except Exception :
                                continue    
                    except Exception :
                        continue                  
            try:
                more = bot.find_element_by_partial_link_text(
                    moreText).get_attribute('href')
                self.get(more)
            
            except Exception as e:
                print(" Can't get more comments")
                break
        return comments,dates

<h4>extractPostandComments</h4> Here we create the output file, extract the posts then the comments and finaly write all the needed data in the outputfile

In [ ]:
def extractPostandComments(bot,url,filename):
    # outputfile creation
    output_file = open (filename,'w',newline='',
                        encoding='utf-8')
    output = csv.writer(output_file, delimiter=',',quotechar='"')      

    # post extraction
    posts,comments_links,post_dates = getPostInGroup(bot,deep=20,url=url)  
    for i in range(len(posts)):
        print ( "post ", i , " out of " , len(posts))
        output.writerow([posts[i],'post',comments_links[i],post_dates[i] ] )
        #comment extraction
        comments,comments_dates = getCommentsInPost(bot,deep=8,url=comments_links[i] )
        for j in range ( len(comments)):
                output.writerow([comments[j],'comment',comments_links[i],comments_dates[j] ] )

    output_file.close()

In [ ]:
if __name__ == '__main__':
    bot = FacebookScrapper()
    try:
        bot.login(input("email : "), getpass("password : "))
    except Exception:
        print( 'already logged ' )
    extractPostandComments(bot,url ="https://mbasic.facebook.com/smugslamarsa/",filename='smugsmarsa.csv')  
    extractPostandComments(bot,url ="https://mbasic.facebook.com/SmugsCoffeeShop/",filename='smugs.csv')